# Introduction

This notebook demonstrates the use of the pre-trained molecular potentials to perform structural relaxations, molecular dynamics simulations and single-point calculations.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu

In [ ]:
from __future__ import annotations

import warnings

from ase.build import molecule
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

import matgl
from matgl.ext._ase_dgl import MolecularDynamics, PESCalculator, Relaxer

# To suppress warnings for clearer output
warnings.simplefilter("ignore")

/Users/kenko/miniconda3/envs/mavrl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading the pre-trained M3GNet PES model

We will first load the M3GNet PES model, which is trained on the subset of ANI-1x dataset. This can be done with a single line of code. Here we only use M3GNet for demonstration and users can choose other available models.

In [ ]:
# You can load any pretrained potentials stored in the 'pretrained_models' directory
# To see available models, use get_available_pretrained_models()
pot = matgl.load_model("M3GNet-ANI-1x-Subset-PES/")

# Structure Relaxation

To perform structure relaxation, we use the Relaxer class. Here, we demonstrate the relaxation of a simple CsCl structure.

In [ ]:
relaxer = Relaxer(potential=pot, relax_cell=False)
atoms = molecule("H2O")
relax_results = relaxer.relax(atoms, fmax=0.01)
# extract results
final_structure = relax_results["final_structure"]
final_energy = relax_results["trajectory"].energies[-1]
# print out the final relaxed structure and energy

print(final_structure)
print(f"The final energy is {float(final_energy):.3f} eV.")

Full Formula (H2 O1)
Reduced Formula: H2O
Charge = 0, Spin Mult = 1
Sites (3)
0 O     0.000000    -0.000000     0.116472
1 H     0.000000     0.760003    -0.475652
2 H     0.000000    -0.760003    -0.475652
The final energy is -2078.627 eV.


# Molecular Dynamics

MD simulations are performed with the ASE interface.

In [ ]:
# Initialize the velocity according to Maxwell Boltzamnn distribution
MaxwellBoltzmannDistribution(atoms, temperature_K=300)
# Create the MD class
driver = MolecularDynamics(atoms, potential=pot, temperature=300, logfile="md_trial.log")
# Run
driver.run(100)
print(f"The potential energy of H2O at 300 K after 100 steps is {float(atoms.get_potential_energy()):.3f} eV.")

The potential energy of H2O at 300 K after 100 steps is -2078.431 eV.


# Single point energy calculation

Perform a single-point calculation for final structure using PESCalculator.

In [ ]:
# define the M3GNet calculator
calc = PESCalculator(pot)
# set up the calculator for atoms object
atoms.set_calculator(calc)
print(f"The calculated potential energy is {atoms.get_potential_energy():.3f} eV.")

The calculated potential energy is -2078.431 eV.
